<a href="https://colab.research.google.com/github/YotamKatzir/YotamKatzir_ImageColourization/blob/main/YotamKatzir_Image_Colourization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from matplotlib import image
from matplotlib import pyplot as plt
import os
from tensorflow import keras
from tensorflow.keras import layers
import time
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

לאימון נשתמש במעבד מסוג יע"ט (TPU - Tensor processing unit)

נגדירו:

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# קוד איתחול היע"ט שצריך להגדיר בתחילת הפרויקט כדי שזה יוכל לעבוד.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

ValueError: ignored

# חלק 1: הכנת הנתונים
בחלק זה אעבור על התמונות הצבעוניות ואעבד אותן לכדי תמונות בגווני אפור.

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/Deep_Learning/PortraitData.zip

In [ ]:
# גודל האצווה לאימון
BATCH_SIZE = 128
BUFFER_SIZE = 60000

# גודל התמונה שאשתמש בו
img_size = 120
# מספר התמונות מתוך המאגר שאשתמש בהן
dataset_split = 6000
# התיקייה שממנה יילקחו התמונות
master_dir = '/content/drive/MyDrive/Deep_Learning/PortraitData'
# הגדרת מערכי הנתונים
x = []
y = []

rgb_check = []
gray_check = []

for image_file in os.listdir(master_dir)[0 : dataset_split]:

  # יצירה ונרמול של התמונות הצבעוניות ובגווני אפור

  rgb_image = Image.open(os.path.join(master_dir, image_file)).resize((img_size, img_size))
  rgb_check.append(rgb_image)
  rgb_img_array = (np.asarray(rgb_image))/255

  gray_image = rgb_image.convert('L')
  gray_check.append(gray_image)
  gray_img_array = (np.asarray(gray_image).reshape((img_size, img_size, 1)))/255

  # שמירת התמונות כמערכי מידע
  x.append(gray_img_array)
  y.append(rgb_img_array)

# פיצול לאימון ומבחן
#train_x, test_x, train_y, test_y = train_test_split(np.array(x), np.array(y), test_size=0.1)

# שמירת המערכים כמאגרי מידע בטנזורפלו
#dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).cache().prefetch(tf.data.experimental.AUTOTUNE).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## נראה שזה עובד

In [ ]:
i = 5

In [ ]:
plt.imshow(rgb_check[i])

In [ ]:
plt.imshow(gray_check[i], cmap='gray')

נציין שזה לא נראה נכון, אבל זה בסדר. אמנם פייפלוט משום מה מציג את התמונה בגווני ירוק במקום בגווני אפור, אבל נראה שמבחינת הרכב מערך התמונה זה בסדר

In [ ]:
pix=np.array(gray_check[i])
np.shape(pix)

# חלק 2: מערכת הרי"ע

במערכת יש שימוש בשתי רשתות עמוקות: האחת, הנקראת המסנן, תלמד לזהות התאמה בין תמונות בגווני אפור למקבילותיהן הצבעוניות, ע"פ מאגר הנתונים שיצרתי. השניה, הנקראת היוצר, או המחולל, תנסה לצייר את ההתאמה הצבעונית לתמונה בשחור לבן עד שהמסנן יחזיר אחוזי התאמה טובים מספיק.

## ראשית, המסנן:

In [ ]:
def disc_model():
  layers = [
    tf.keras.layers.Conv2D(32, kernel_size=(7, 7), strides=1, activation='relu', kernel_regularizer='L2', input_shape=(120, 120, 3)),
    tf.keras.layers.Conv2D(32, kernel_size=(7, 7), strides=1, activation='relu'),

    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(64, kernel_size=(5, 5), kernel_regularizer='L2', strides=1, activation='relu'),
    tf.keras.layers.Conv2D(64, kernel_size=(5, 5), strides=1, activation='relu'),
    
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), kernel_regularizer='L2', strides=1, activation='relu'),
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1, activation='relu'),
    
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), kernel_regularizer='L2', strides=1, activation='relu'),
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1, activation='relu'),
    
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
      ]
  model = tf.keras.models.Sequential(layers, name="Discriminator")
  return model

In [ ]:
with strategy.scope():
  DiscModel = disc_model()
DiscModel.summary()

## כעת, היוצר:

In [ ]:
def gen_model():
  
    inputs = tf.keras.layers.Input(shape=(img_size, img_size, 1))

    conv1 = tf.keras.layers.Conv2D(16, kernel_size=(5, 5), strides=1)(inputs)
    conv1 = tf.keras.layers.LeakyReLU()( conv1 )
    conv1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1)(conv1)
    conv1 = tf.keras.layers.LeakyReLU()(conv1)
    conv1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1)(conv1)
    conv1 = tf.keras.layers.LeakyReLU()(conv1)

    conv2 = tf.keras.layers.Conv2D(32, kernel_size=(5, 5), strides=1)(conv1)
    conv2 = tf.keras.layers.LeakyReLU()(conv2)
    conv2 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1)(conv2)
    conv2 = tf.keras.layers.LeakyReLU()(conv2)
    conv2 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1)(conv2)
    conv2 = tf.keras.layers.LeakyReLU()(conv2)

    conv3 = tf.keras.layers.Conv2D(64, kernel_size=(5, 5) ,strides=1)(conv2)
    conv3 = tf.keras.layers.LeakyReLU()(conv3)
    conv3 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1)(conv3)
    conv3 = tf.keras.layers.LeakyReLU()(conv3)
    conv3 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1)(conv3)
    conv3 = tf.keras.layers.LeakyReLU()(conv3)

    bottleneck = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1, activation='tanh', padding='same')(conv3)

    concat_1 = tf.keras.layers.Concatenate()([bottleneck, conv3])
    conv_up_3 = tf.keras.layers.Conv2DTranspose(128, kernel_size=(3, 3), strides=1, activation='relu')(concat_1)
    conv_up_3 = tf.keras.layers.Conv2DTranspose(128, kernel_size=(3, 3), strides=1, activation='relu')(conv_up_3)
    conv_up_3 = tf.keras.layers.Conv2DTranspose(64, kernel_size=(5, 5), strides=1, activation='relu')(conv_up_3)

    concat_2 = tf.keras.layers.Concatenate()([conv_up_3, conv2])
    conv_up_2 = tf.keras.layers.Conv2DTranspose(64, kernel_size=(3, 3), strides=1, activation='relu')(concat_2)
    conv_up_2 = tf.keras.layers.Conv2DTranspose(64, kernel_size=(3, 3), strides=1, activation='relu')(conv_up_2)
    conv_up_2 = tf.keras.layers.Conv2DTranspose(32, kernel_size=(5, 5), strides=1, activation='relu')(conv_up_2)

    concat_3 = tf.keras.layers.Concatenate()([conv_up_2, conv1])
    conv_up_1 = tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=1, activation='relu')(concat_3)
    conv_up_1 = tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=1, activation='relu')(conv_up_1)
    conv_up_1 = tf.keras.layers.Conv2DTranspose(3, kernel_size=(5, 5), strides=1, activation='relu')(conv_up_1)

    model = tf.keras.models.Model(inputs, conv_up_1, name="Generator")
    return model

In [ ]:
with strategy.scope():
  GenModel = gen_model()
GenModel.summary()

## והרי פונקציות האובדן של הרשתות:

In [ ]:
# הגדרת פונקציות האובדן אנטרופיה מוצלבת וממוצעים ריבועיים משוקללים
cross_entropy = tf.keras.losses.BinaryCrossentropy()
mse = tf.keras.losses.MeanSquaredError()

# הגדרת אובדן המסנן
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output) - tf.random.uniform( shape=real_output.shape , maxval=0.1 ) , real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output) + tf.random.uniform( shape=fake_output.shape , maxval=0.1  ) , fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# הגדרת אובדן היוצר
def generator_loss(fake_output , real_y):
    real_y = tf.cast( real_y , 'float32' )
    return mse( fake_output , real_y )

# נגדיר את האופטימזציות של היצרן והמסנן להיות אדם
generator_optimizer = tf.keras.optimizers.Adam(0.0005)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0005)

# ניצור את פונקציות היצרן והמסנן
generator = gen_model()
discriminator = disc_model()

# חלק 3: אימון

In [ ]:
EPOCHS = 140

In [ ]:
@tf.function
def train_step(input_x, real_y):

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    # יצירת תמונה על ידי היוצר
    generated_images = generator(input_x , training=True)
    # ההסתברות שהתמונה הנתונה היא האמיתית לפי המסנן
    real_output = discriminator(real_y, training=True)
    # ההסתברות שהתמונה שיוצרה היא האמתית לפיו
    generated_output = discriminator(generated_images, training=True)
    
    # נגדיר את אובדני המסנן והיוצר
    gen_loss = generator_loss(generated_images, real_y)
    disc_loss = discriminator_loss(real_output, generated_output)

  tf.keras.backend.print_tensor(tf.keras.backend.mean(gen_loss))
  tf.keras.backend.print_tensor (tf.keras.backend.mean(disc_loss))
  tf.keras.backend.print_tensor(gen_loss+disc_loss)

  # חישוב הנגזרות
  grad_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  grad_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

  # אופטימיזציה עם אדם
  generator_optimizer.apply_gradients(zip(grad_generator, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(grad_discriminator, discriminator.trainable_variables)) 

In [ ]:
num_epochs = EPOCHS

for i in range( num_epochs ):
    print( i )
    for ( x , y ) in dataset:
        # איקס ו-וואי מייצגים אוצוות מידע
        print( x.shape )
        train_step( x , y )

# חלק 4 – תוצאות

In [ ]:
y = generator(test_x[0:]).numpy()

In [ ]:
for i in range(len(test_x)):
  plt.figure(figsize=(10,10))
  or_image = plt.subplot(3,3,1)
  or_image.set_title('Grayscale Input', fontsize=16)
  plt.imshow( test_x[i].reshape((120,120)) , cmap='gray' )

  in_image = plt.subplot(3,3,2)    
  image = Image.fromarray( ( y[i] * 255 ).astype( 'uint8' ) ).resize( ( 1024 , 1024 ) )
  image = np.asarray( image )
  in_image.set_title('Colorized Output', fontsize=16)
  plt.imshow( image )

  ou_image = plt.subplot(3,3,3)
  image = Image.fromarray( ( test_y[i] * 255 ).astype( 'uint8' ) ).resize( ( 1024 , 1024 ) )
  ou_image.set_title('Ground Truth', fontsize=16)
  plt.imshow( image )

  plt.show()